 <h1 style="text-align: center; font-size: 50px;font-weight: bold;"> Bayesian Machine Learning
   </h1>


 <h1 style="text-align: center; font-size: 25px;font-weight: bold;"> Project : Computing Nonvacuous Generalization Bounds for Deep (Stochastic) Neural
 Networks with Many More Parameters than Training Data </h1>


In [ ]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.init as init
import numpy as np
import torch.optim as optim

import math
import time as ts

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
# Download and load the dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# train_dataset = torch.utils.data.Subset(train_dataset, range(55000))
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

print(train_dataset)
print(test_dataset)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

# Get a batch of images and labels
images, labels = next(iter(train_loader))

In [ ]:
print(images.shape, labels.shape)

In [ ]:
plt.imshow(images[0,0,:,:])
print(torch.mean(images[0,0,:,:]), torch.max(images[0,0,:,:]), torch.min(images[0,0,:,:]))

In [ ]:
class MLP(nn.Module):
    def __init__(self, nb_nodes, nb_hidden_layers, classes=1):
        super().__init__()

        layers = [torch.nn.Flatten(), nn.Linear(28 * 28, nb_nodes), nn.ReLU()]

        for _ in range(nb_hidden_layers - 1):
            layers.append(nn.Linear(nb_nodes, nb_nodes))
            layers.append(nn.ReLU())
            # layers.append(nn.Dropout(p=0.4))

        layers.append(nn.Linear(nb_nodes, classes))

        self.layers = nn.Sequential(*layers)

        self.initialize_weights()   # Initialize weights after layer creation

    def forward(self, input):
        x = torch.flatten(input, start_dim=1)
        x = self.layers(x)
        return x

    def truncated_normal_(self, tensor, mean=0, std=0.04, low=-2, high=2):
        """Truncated normal initialization for weights."""
        with torch.no_grad():
            tensor.normal_(mean, std)
            tensor.clamp_(low*std, high*std)  # Truncate values to [-2*sigma, 2*sigma]

    def initialize_weights(self):
        """Initialize weights and biases as per the given specifications."""
        linear_layer = 0
        for  m in self.modules():
            if isinstance(m, nn.Linear):
                self.truncated_normal_(m.weight, mean=0, std=0.04, low=-2, high=2)
                
                if linear_layer == 0:  # First layer after input
                    init.constant_(m.bias, 0.1)
                else:
                    init.constant_(m.bias, 0)
                linear_layer +=1

In [ ]:
lr = 0.0001
epochs = 200
patience = 20

In [ ]:
import copy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = MLP(nb_nodes=600, nb_hidden_layers=2).to(device)
model = MLP(nb_nodes=600, nb_hidden_layers=1).to(device)
print(model)

# optimizer = torch.optim.RMSprop(model.parameters(), lr = lr, momentum=0.9, eps=1e-5)
# optimizer = torch.optim.Adam(model.parameters(), lr = lr)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
def print_weights_and_biases(model):
    for name, param in model.named_parameters():
        if "weight" in name:
            print(f"Layer: {name} | Weights:\n{param.data}\n")
        elif "bias" in name:
            print(f"Layer: {name} | Biases:\n{param.data}\n")

# print_weights_and_biases(model)
# path = '/kaggle/working/'
path = ''

In [ ]:
import torch.nn.functional as F

def plot_training_history(history_train_loss, history_val_loss):
    epochs = range(1, len(history_train_loss) + 1)

    # figure avec deux sous-graphes côte à côte
    plt.figure(figsize=(14, 5))

    # Graphique pour la perte
    plt.semilogy(epochs, history_train_loss, label='Train Loss', marker='o')
    plt.semilogy(epochs, history_val_loss, label='Validation Loss', marker='o')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

def label_modif(label):
    """ Changes the labels of the MNIST dataset to transform it into a binary
    classification problem by mapping numbers {0,.....,4} to label 1 and {5,......,9} to label -1"""
    # print("label check before", label)
    label[label<=4] = 1
    label[label >=5] = -1
    # print("label check after", label)
    
    return label

def logistic_loss(labels, outputs):
    loss = (1/(np.log(2)))*torch.log(1+torch.exp(-labels*outputs))
    return torch.mean(loss)


def training_pipeline(epochs, model, train_loader, test_loader, optimizer, patience):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loss_train_tab = []
    loss_valid_tab = []
    best_val_loss = float("inf")
    best_model = None
    model.to(device)
    print("Module sélectionné pour l'entraînement :", device)
    epochs_no_improve = 0
    
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0.0
        total_train_correct = 0
        total_train_samples = 0
        
        for k, (image_train, label_train) in enumerate(train_loader):
            label_train = label_modif(label_train)
            image_train, label_train = image_train.to(device), label_train.to(device).float()
            optimizer.zero_grad()
            output = model(image_train)
            loss = logistic_loss(label_train, output.squeeze(1))
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
            # Prediction: sign of output as predicted label (assumes output is a real number)
            predictions = torch.sign(output.squeeze(1))
            total_train_correct += (predictions == label_train).sum().item()
            total_train_samples += label_train.size(0)
        
        avg_train_loss = total_train_loss / len(train_loader)
        train_accuracy = total_train_correct / total_train_samples
        loss_train_tab.append(avg_train_loss)
        
        model.eval()
        total_val_loss = 0.0
        total_val_correct = 0
        total_val_samples = 0
        with torch.no_grad():
            for k, (image_test, label_test) in enumerate(test_loader):
                label_test = label_modif(label_test)
                image_test, label_test = image_test.to(device), label_test.to(device).float()
                output = model(image_test)
                loss = logistic_loss(label_test, output.squeeze(1))
                total_val_loss += loss.item()
                predictions = torch.sign(output.squeeze(1))
                total_val_correct += (predictions == label_test).sum().item()
                total_val_samples += label_test.size(0)
        avg_val_loss = total_val_loss / len(test_loader)
        val_accuracy = total_val_correct / total_val_samples
        loss_valid_tab.append(avg_val_loss)
        
        print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model = copy.deepcopy(model)  # Sauvegarde du meilleur modèle
            torch.save(best_model.state_dict(), path + 'model_init.pt')
            
            epochs_no_improve = 0  # Reset du compteur
            print(f"A new best model has been found with validation loss: {best_val_loss:.4f}")
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {patience} epochs with no improvement.")
            break
        
        if (epoch + 1) % 10 == 0:
            plot_training_history(loss_train_tab, loss_valid_tab)
            
    plot_training_history(loss_train_tab, loss_valid_tab)
    return best_model, loss_valid_tab

        

In [ ]:
best_model, loss_validation = training_pipeline(epochs=20, model=model, train_loader=train_loader, test_loader=test_loader, optimizer=optimizer, patience=patience)

# PAC BOUND

In [ ]:
# Stochastic Linear Layer

class StochLinear(torch.nn.Module):
    def __init__(self, in_features, out_features, bias=True, 
                 init_inv_softplus_sigma=-3.0, eps=1e-6):
        super().__init__()
        self.weight_mu = torch.nn.Parameter(torch.zeros(out_features, in_features))
        self.weight_ispsigma = torch.nn.Parameter(torch.empty(out_features, in_features).fill_(
                                                                                init_inv_softplus_sigma))
        if bias:
            self.bias_mu = torch.nn.Parameter(torch.zeros(out_features))
            self.bias_ispsigma = torch.nn.Parameter(torch.empty(out_features).fill_(init_inv_softplus_sigma))
        self.eps = eps
        self.with_bias = bias
    
    
    def forward(self, x):
        weight = self.weight_mu + torch.randn_like(self.weight_mu) * self.weight_sigma
        if self.with_bias:
            bias = self.bias_mu + torch.randn_like(self.bias_mu) * self.bias_sigma
        else:
            bias = None
        return torch.nn.functional.linear(x, weight, bias)
    
    def copy_weights(self, linear_layer):
        self.weight_mu.detach().copy_(linear_layer.weight)
        if self.with_bias:
            self.bias_mu.detach().copy_(linear_layer.bias)
            
    @property
    def weight_sigma(self):
        return torch.nn.functional.softplus(self.weight_ispsigma) + self.eps
    
    @property
    def bias_sigma(self):
        return torch.nn.functional.softplus(self.bias_ispsigma) + self.eps
            

In [ ]:
# Stochastic Neural Network

class StochaNN(nn.Module):
    def __init__(self, nb_nodes, nb_hidden_layers, classes=1, isp_sigma=-3.0):
        super().__init__()

        layers = [torch.nn.Flatten(), StochLinear(28*28, nb_nodes, init_inv_softplus_sigma=isp_sigma), nn.ReLU()]

        for _ in range(nb_hidden_layers - 1):
            layers.append(StochLinear(nb_nodes, 600, init_inv_softplus_sigma=isp_sigma))
            layers.append(nn.ReLU())
            # layers.append(nn.Dropout(p=0.4))

        layers.append(StochLinear(nb_nodes, classes, init_inv_softplus_sigma=isp_sigma))

        self.layers = nn.Sequential(*layers)

    def forward(self, input):
        return self.layers(input)

In [ ]:
# Approximate PAC-Bayes bound

def kl_bernoulli(q, p):
    return q * math.log(q / p)+ (1 - q) * math.log((1 - q) / (1 - p))
    
def get_approx_bound(b_re, acc, res=1000):
    ts = np.linspace(0., 1., res)
    print("acc", acc)
    kls = [kl_bernoulli(1 - acc, t) for t in ts]
    idx = np.argmin(np.abs(np.array(kls) - b_re))
    return ts[idx]

# Training
def train_epoch(loader, model, loss_fn, optimizer, device=device):
    """
    Runs one training epoch.
    """
    model.train()
    running_loss = 0.0
    total_samples = 0
    total_train_samples = 0
    total_train_correct = 0
    acc = 0

    
    for i, (inputs, targets) in enumerate(loader):
    
        inputs = inputs.to(device)
        targets = label_modif(targets)
        targets = targets.to(device).float()
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss, _, _ = loss_fn(model, inputs, targets)
        # print(loss)
        loss.backward()
        optimizer.step()
        
        batch_size = inputs.size(0)
        running_loss += loss.item() * batch_size
        total_samples += batch_size
        total_train_samples += batch_size
        predictions = torch.sign(outputs.squeeze(1))
        total_train_correct += (predictions == targets).sum().item()

        total_train_samples += 1
    acc = acc / len(loader)
    avg_loss = running_loss / total_samples
    train_accuracy = total_train_correct / total_train_samples
    
    return {"loss": avg_loss, "stats": {'accuracy': train_accuracy}}


def eval_model(loader, model, loss_fn, device=device):
    """
    Evaluates the model on the given dataset.
    """
    model.eval()
    running_loss = 0.0
    total_samples = 0
    total_correct = 0
    acc = 0
    
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(loader):
            inputs = inputs.to(device)
            targets = label_modif(targets)
            targets = targets.to(device).float()
            outputs = model(inputs)
            loss, _, _ = loss_fn(model, inputs, targets)
            batch_size = inputs.size(0)
            running_loss += loss.item() * batch_size
            total_samples += batch_size
            predictions = torch.sign(outputs.squeeze(1))
            total_correct += (predictions == targets).sum().item()
        
        acc = acc / len(loader)
        avg_loss = running_loss / total_samples
        accuracy = total_correct / total_samples
    return {"loss": avg_loss, "stats": {'accuracy': accuracy}}


In [ ]:
class PACBound:
    def __init__(self, prior_mean, lambda_init, likelihood_fn, num_data, delta=0.025, b=100, c=0.1):
        
        self.prior_mu = prior_mean.to(device)
        self.log_lambda_ = torch.tensor([math.log(lambda_init)],
                                   requires_grad=True, device=device)
        self.likelihood_fn = likelihood_fn
        self.num_data = torch.tensor([num_data], requires_grad=False, 
                                     dtype=float, device=device)
        self.delta = delta
        self.b = b
        self.c = c
        
    def kl(self, stoch_network):
        mu_net, sigma_net = self._get_mu_sigma(stoch_network.layers)
        prior_sigma = torch.ones_like(sigma_net, device=sigma_net.device) * self.prior_sigma.cuda()

        return self._kl_gaussians(mu_net, sigma_net, self.prior_mu, prior_sigma)
    
    def train_bound(self, stoch_network, x, y):
        preds = stoch_network(x).flatten()
        acc = (preds * y > 0).float().mean()
        likelihood = self.likelihood_fn(preds, y)
        regularizer =  torch.sqrt(self.B_RE(stoch_network) / 2)
        # print(likelihood, regularizer)
        return likelihood + regularizer, preds, \
               {'accuracy': acc, 'regularizer': regularizer}
        
    def B_RE(self, stoch_network):
        kl = self.kl(stoch_network)
        return (kl + 
                2 * torch.log(torch.abs(self.b * (math.log(self.c) - self.log_lambda_))) + 
                math.log(math.pi**2 * self.num_data / (6 * self.delta))) / (self.num_data - 1)
        
    @property
    def prior_sigma(self):
        return 1 / torch.sqrt(torch.exp(self.log_lambda_))
        
    @staticmethod
    def _kl_gaussians(mu_1, sigma_1, mu_2, sigma_2):
        device = mu_1.device

        mu_1 = mu_1.to(device)
        mu_2 = mu_2.to(device)
        sigma_1 = sigma_1.to(device)
        sigma_2 = sigma_2.to(device)
        
        d = mu_1.numel()
        kl = 0.5 * (torch.sum(torch.log(sigma_2) - torch.log(sigma_1)) * 2 +
                    torch.sum(sigma_1**2 / sigma_2**2) +
                    torch.sum((mu_2 - mu_1)**2 / sigma_2**2) - d)
        return kl 
        
    @staticmethod
    def _get_mu_sigma(stoch_network):
        mu_net, sigma_net = [torch.cat(a) for a in 
                             zip(*[
                                  [torch.cat([layer.weight_mu.flatten(), layer.bias_mu.flatten()]),
                                  torch.cat([layer.weight_sigma.flatten(), layer.bias_sigma.flatten()])]
                                  for 
                                  layer in stoch_network if
                                  isinstance(layer, StochLinear)])
                            ]
        return mu_net, sigma_net

In [ ]:
model = MLP(nb_nodes=600, nb_hidden_layers=1)
state_dict = torch.load(path + "model_init.pt")
model.load_state_dict(state_dict)
model.cuda()

prior_mu = torch.cat([p.detach().flatten() for p in model.layers.parameters()])
pac_bound = PACBound(prior_mu, lambda_init=1e-3, likelihood_fn=logistic_loss, num_data=len(train_dataset))

stoch_model = StochaNN(nb_nodes=600, nb_hidden_layers=1, isp_sigma=-2)
print(stoch_model)
for stoch_layer, layer in zip(stoch_model.layers, model.layers):
    print("stoch", stoch_layer, "layer", layer)
    if isinstance(stoch_layer, StochLinear):
        stoch_layer.copy_weights(layer)
stoch_model.cuda()

In [ ]:
import time as ts

# Number of epochs to match the paper (200,000 iterations)

epochs = 1000
initial_lr = 0.001  # First 150,000 iterations
lr_drop_epoch = 250  # Drop learning rate at 150,000 iterations
lr_factor = 0.1  # Reduce learning rate by lr_factor
eval_freq = 100  # Evaluate every 100 epochs

# Initialize RMSprop optimizer (with alpha=0.9 as in the paper)
optimizer_sto = torch.optim.RMSprop([
    {'params': stoch_model.parameters(), 'lr': initial_lr, 'alpha': 0.9}, 
    {'params': pac_bound.log_lambda_, 'lr': initial_lr}
])

lambdas, train_accs, test_accs, b_re_values, kl_values, bounds, idx = [], [], [], [], [], [], []

for epoch in range(epochs):
    _ts = ts.perf_counter()

    # Drop learning rate after 250 epochs
    if epoch == lr_drop_epoch:
        for param_group in optimizer_sto.param_groups:
            param_group['lr'] *= lr_factor
    
    # Run one training epoch
    train_res = train_epoch(train_loader, stoch_model, pac_bound.train_bound, optimizer_sto)
    if epoch % eval_freq == 0:
        # Evaluate the model on the test set
        eval_res = eval_model(test_loader, stoch_model, pac_bound.train_bound)
        test_accs.append(eval_res['stats']['accuracy'])
        idx.append(epoch)
        print(f'**** Epoch {epoch + 1} / {epochs}: Test accuracy: {test_accs[-1]} ****')
    train_accs.append(train_res['stats']['accuracy'])
    
    # Compute PAC-Bayes metrics
    lambdas.append(torch.exp(pac_bound.log_lambda_).item())
    b_re_values.append(pac_bound.B_RE(stoch_model).item())

    stoch_model.eval()
    kl = pac_bound.kl(stoch_model).item()
    kl_values.append(kl)
    bounds.append(get_approx_bound(b_re_values[-1], train_accs[-1]))
    
    print(f"Epoch {epoch + 1} / {epochs}: Train Acc = {train_accs[-1] :.4f}, λ = {lambdas[-1] :.4f}, "
          f"Bound:{bounds[-1] :.4f}, B_RE = {b_re_values[-1] :.4f}, KL = {kl_values[-1] :.4f}, Time: {ts.perf_counter() - _ts :.2f}s")


# Plots
f, arr = plt.subplots(1, 4, figsize=(18, 6))

arr[0].plot(train_accs, lw=2, label="Train Accuracy", color="blue")
arr[0].plot(idx, test_accs, lw=2, label="Test Accuracy", color="red")
arr[0].set_ylabel("Accuracy", fontsize=14)
arr[0].set_xlabel("Epoch", fontsize=14)
arr[0].legend()
arr[0].grid()

arr[1].plot(lambdas, lw=2, label=r"$\lambda$", color="red")
arr[1].set_ylabel(r"$\lambda$", fontsize=14)
arr[1].set_xlabel("Epoch", fontsize=14)
arr[1].legend()
arr[1].grid()

# Plot KL Divergence
arr[2].plot(kl_values, lw=2, label="KL Divergence", color="green")
arr[2].set_ylabel("KL Divergence", fontsize=14)
arr[2].set_xlabel("Epoch", fontsize=14)
arr[2].legend()
arr[2].grid()

# Plot Bound
arr[3].plot(bounds, lw=2, label="Bound", color="green")
arr[3].set_ylabel("Bound", fontsize=14)
arr[3].set_xlabel("Epoch", fontsize=14)
arr[3].legend()
arr[3].grid()

plt.tight_layout()
plt.show()

In [ ]:
stoch_model

In [ ]:
torch.save(stoch_model.state_dict(), path + "stoch_model.pt")
# torch.save(pac_bound.log_lambda_, "lambda.pt")

# Multi Class

In [ ]:
def training_pipeline_multi(epochs, model, train_loader, test_loader, optimizer, patience):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loss_train_tab = []
    loss_valid_tab = []
    best_val_loss = float("inf")
    best_model = None
    model.to(device)
    print("Module sélectionné pour l'entraînement :", device)
    epochs_no_improve = 0
    criterion = torch.nn.CrossEntropyLoss()  # Using CrossEntropyLoss for multi-class classification

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0.0
        total_train_correct = 0
        total_train_samples = 0
        
        for k, (image_train, label_train) in enumerate(train_loader):
            image_train, label_train = image_train.to(device), label_train.to(device).long()
            optimizer.zero_grad()
            output = model(image_train)  
            loss = criterion(output, label_train)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
            
            # Get predictions: max value index as the predicted class
            predictions = torch.argmax(output, dim=1)  # Predictions are the class with the highest logit
            total_train_correct += (predictions == label_train).sum().item()
            total_train_samples += label_train.size(0)
        
        avg_train_loss = total_train_loss / len(train_loader)
        train_accuracy = total_train_correct / total_train_samples
        loss_train_tab.append(avg_train_loss)
        
        model.eval()
        total_val_loss = 0.0
        total_val_correct = 0
        total_val_samples = 0
        with torch.no_grad():
            for k, (image_test, label_test) in enumerate(test_loader):
                image_test, label_test = image_test.to(device), label_test.to(device).long() 
                output = model(image_test)
                loss = criterion(output, label_test)
                total_val_loss += loss.item()
                
                # Get predictions: max value index as the predicted class
                predictions = torch.argmax(output, dim=1)  # Predictions are the class with the highest logit
                total_val_correct += (predictions == label_test).sum().item()
                total_val_samples += label_test.size(0)
        
        avg_val_loss = total_val_loss / len(test_loader)
        val_accuracy = total_val_correct / total_val_samples
        loss_valid_tab.append(avg_val_loss)
        
        print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model = copy.deepcopy(model)  # Save the best model
            epochs_no_improve = 0  # Reset the counter
            print(f"A new best model has been found with validation loss: {best_val_loss:.4f}")
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {patience} epochs with no improvement.")
            break
        
        if (epoch + 1) % 10 == 0:
            plot_training_history(loss_train_tab, loss_valid_tab)
            
    plot_training_history(loss_train_tab, loss_valid_tab)
    return best_model, loss_valid_tab

In [ ]:
model_multiclass = MLP(nb_nodes=600, nb_hidden_layers=2, classes=10).to(device)
optimizer_multi = torch.optim.SGD(model_multiclass.parameters(), lr=0.01, momentum=0.9)
best_model, loss_valid_tab = training_pipeline_multi(epochs=20, model=model_multiclass, train_loader=train_loader, test_loader=test_loader, optimizer=optimizer_multi, patience=patience)

In [ ]:
torch.save(model_multiclass.state_dict(), path + "model_init_multiclass.pt")

In [ ]:
def train_epoch(loader, model, loss_fn, optimizer, device=device):
    """
    Runs one training epoch.
    """
    model.train()
    running_loss = 0.0
    total_samples = 0
    total_train_samples = 0
    total_train_correct = 0
    acc = 0

    
    for i, (inputs, targets) in enumerate(loader):
    
        inputs = inputs.to(device)
        # targets = label_modif(targets)
        targets = targets.to(device).float()
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss, _, _ = loss_fn(model, inputs, targets)
        # print(loss)
        loss.backward()
        optimizer.step()
        
        batch_size = inputs.size(0)
        running_loss += loss.item() * batch_size
        total_samples += batch_size
        total_train_samples += batch_size
        predictions = torch.argmax(outputs, dim=1)
        total_train_correct += (predictions == targets).sum().item()

        total_train_samples += 1
    acc = acc / len(loader)
    avg_loss = running_loss / total_samples
    train_accuracy = total_train_correct / total_train_samples
    
    return {"loss": avg_loss, "stats": {'accuracy': train_accuracy}}


def eval_model(loader, model, loss_fn, device=device):
    """
    Evaluates the model on the given dataset.
    """
    model.eval()
    running_loss = 0.0
    total_samples = 0
    total_correct = 0
    acc = 0
    
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(loader):
            inputs = inputs.to(device)
            targets = targets.to(device).long()
            outputs = model(inputs)
            loss, _, _ = loss_fn(model, inputs, targets)
            batch_size = inputs.size(0)
            running_loss += loss.item() * batch_size
            total_samples += batch_size
            predictions = torch.argmax(outputs, dim=1)
            total_correct += (predictions == targets).sum().item()
        
        acc = acc / len(loader)
        avg_loss = running_loss / total_samples
        accuracy = total_correct / total_samples
    return {"loss": avg_loss, "stats": {'accuracy': accuracy}}

In [ ]:
class PACBound:
    def __init__(self, prior_mean, lambda_init, likelihood_fn, num_data, delta=0.025, b=100, c=0.1):
        
        self.prior_mu = prior_mean.to(device)
        self.log_lambda_ = torch.tensor([math.log(lambda_init)],
                                   requires_grad=True, device=device)
        self.likelihood_fn = likelihood_fn
        self.num_data = torch.tensor([num_data], requires_grad=False, 
                                     dtype=float, device=device)
        self.delta = delta
        self.b = b
        self.c = c
        
    def kl(self, stoch_network):
        mu_net, sigma_net = self._get_mu_sigma(stoch_network.layers)
        prior_sigma = torch.ones_like(sigma_net, device=sigma_net.device) * self.prior_sigma.cuda()

        return self._kl_gaussians(mu_net, sigma_net, self.prior_mu, prior_sigma)

    
    # def train_bound(self, stoch_network, x, y):
    #     preds = stoch_network(x).flatten()
    #     acc = (preds * y > 0).float().mean()
    #     likelihood = self.likelihood_fn(preds, y)
    #     regularizer =  torch.sqrt(self.B_RE(stoch_network) / 2)
    #     # print(likelihood, regularizer)
    #     return likelihood + regularizer, preds, \
    #            {'accuracy': acc, 'regularizer': regularizer}
    
    def train_bound(self, stoch_network, x, y):
        # Forward pass through the stochastic network
        preds = stoch_network(x)  # This will be of shape (batch_size, num_classes)
        
        # Multi-class accuracy: get the index of the maximum output (i.e., predicted class)
        _, predicted_classes = torch.max(preds, dim=1)
        
        # Compute accuracy
        acc = (predicted_classes == y).float().mean()

        # Compute likelihood (CrossEntropyLoss is used)
        likelihood = self.likelihood_fn(preds, y.long())

        # Compute the regularizer term based on the B_RE bound
        regularizer = torch.sqrt(self.B_RE(stoch_network) / 2)

        return likelihood + regularizer, preds, {'accuracy': acc, 'regularizer': regularizer}
    
    def B_RE(self, stoch_network):
        kl = self.kl(stoch_network)
        return (kl + 
                2 * torch.log(torch.abs(self.b * (math.log(self.c) - self.log_lambda_))) + 
                math.log(math.pi**2 * self.num_data / (6 * self.delta))) / (self.num_data - 1)
        
    @property
    def prior_sigma(self):
        return 1 / torch.sqrt(torch.exp(self.log_lambda_))
        
    @staticmethod
    def _kl_gaussians(mu_1, sigma_1, mu_2, sigma_2):
        device = mu_1.device

        mu_1 = mu_1.to(device)
        mu_2 = mu_2.to(device)
        sigma_1 = sigma_1.to(device)
        sigma_2 = sigma_2.to(device)
        
        d = mu_1.numel()
        kl = 0.5 * (torch.sum(torch.log(sigma_2) - torch.log(sigma_1)) * 2 +
                    torch.sum(sigma_1**2 / sigma_2**2) +
                    torch.sum((mu_2 - mu_1)**2 / sigma_2**2) - d)
        return kl 
        
    @staticmethod
    def _get_mu_sigma(stoch_network):
        mu_net, sigma_net = [torch.cat(a) for a in 
                             zip(*[
                                  [torch.cat([layer.weight_mu.flatten(), layer.bias_mu.flatten()]),
                                  torch.cat([layer.weight_sigma.flatten(), layer.bias_sigma.flatten()])]
                                  for 
                                  layer in stoch_network if
                                  isinstance(layer, StochLinear)])
                            ]
        return mu_net, sigma_net

In [ ]:
model_multiclass = MLP(nb_nodes=600, nb_hidden_layers=2, classes=10)
state_dict = torch.load("model_init_multiclass.pt")
model_multiclass.load_state_dict(state_dict)
model_multiclass.cuda()
crtierion = torch.nn.CrossEntropyLoss()
prior_mu = torch.cat([p.detach().flatten() for p in model_multiclass.layers.parameters()])
pac_bound = PACBound(prior_mu, lambda_init=1e-3, likelihood_fn=crtierion, num_data=len(train_dataset))

stoch_model_multi = StochaNN(nb_nodes=600, nb_hidden_layers=1, classes=10, isp_sigma=-2)
for stoch_layer, layer in zip(stoch_model_multi.layers, model_multiclass.layers):
    if isinstance(stoch_layer, StochLinear):
        stoch_layer.copy_weights(layer)
stoch_model_multi.cuda()

In [ ]:
import time as ts

# Number of epochs to match the paper (200,000 iterations)

epochs = 4
initial_lr = 0.001  # First 150,000 iterations
lr_drop_epoch = 250  # Drop learning rate at 150,000 iterations
lr_factor = 0.1  # Reduce learning rate by lr_factor
eval_freq = 2

# Initialize RMSprop optimizer (with alpha=0.9 as in the paper)
optimizer_sto = torch.optim.RMSprop([
    {'params': stoch_model_multi.parameters(), 'lr': initial_lr, 'alpha': 0.9}, 
    {'params': pac_bound.log_lambda_, 'lr': initial_lr}
])

lambdas, train_accs, test_accs, b_re_values, kl_values, bounds = [], [], [], [], [], []
idx = []

for epoch in range(epochs):
    _ts = ts.perf_counter()

    # Drop learning rate after 250 epochs
    if epoch == lr_drop_epoch:
        for param_group in optimizer_sto.param_groups:
            param_group['lr'] *= lr_factor

    # Run one training epoch
    train_res = train_epoch(train_loader, stoch_model_multi, pac_bound.train_bound, optimizer_sto)
    if epoch % eval_freq == 0:
        idx.append(epoch)
        test_res = eval_model(test_loader, stoch_model_multi, pac_bound.train_bound)
        test_accs.append(test_res['stats']['accuracy'])
        print(f'**** Epoch {epoch + 1} / {epochs}: Test accuracy: {test_accs[-1]} ****')
    train_accs.append(train_res['stats']['accuracy'])
    
    # Compute PAC-Bayes metrics
    lambdas.append(torch.exp(pac_bound.log_lambda_).item())
    b_re_values.append(pac_bound.B_RE(stoch_model_multi).item())

    stoch_model_multi.eval()
    kl = pac_bound.kl(stoch_model_multi).item()
    kl_values.append(kl)
    bounds.append(get_approx_bound(b_re_values[-1], train_accs[-1]))
    
    print(f"Epoch {epoch + 1} / {epochs}: Train Acc = {train_accs[-1] :.4f}, λ = {lambdas[-1] :.4f}, "
          f"Bound:{bounds[-1] :.4f}, B_RE = {b_re_values[-1] :.4f}, KL = {kl_values[-1] :.4f}, Time: {ts.perf_counter() - _ts :.2f}s")


# Plots
f, arr = plt.subplots(1, 4, figsize=(18, 6))

arr[0].plot(train_accs, lw=2, label="Train Accuracy", color="blue")
arr[0].plot(idx, test_accs, lw=2, label="Test Accuracy", color="red")
arr[0].set_ylabel("Accuracy", fontsize=14)
arr[0].set_xlabel("Epoch", fontsize=14)
arr[0].legend()
arr[0].grid()

arr[1].plot(lambdas, lw=2, label=r"$\lambda$", color="red")
arr[1].set_ylabel(r"$\lambda$", fontsize=14)
arr[1].set_xlabel("Epoch", fontsize=14)
arr[1].legend()
arr[1].grid()

# Plot KL Divergence
arr[2].plot(kl_values, lw=2, label="KL Divergence", color="green")
arr[2].set_ylabel("KL Divergence", fontsize=14)
arr[2].set_xlabel("Epoch", fontsize=14)
arr[2].legend()
arr[2].grid()

# Plot Bound
arr[3].plot(bounds, lw=2, label="Bound", color="green")
arr[3].set_ylabel("Bound", fontsize=14)
arr[3].set_xlabel("Epoch", fontsize=14)
arr[3].legend()
arr[3].grid()

plt.tight_layout()
plt.show()